<a href="https://colab.research.google.com/github/EggPudding/Deep-Learning-Practice-with-Codes/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Going Deeper with Convolutions (CVPR 2015) Tutorial**

*   Pratice for GoogLeNet Architecture
*   Orginal Paper: https://arxiv.org/abs/1512.03385
*   Note that you first change **Runtime** to **GPU** setting
*   **CIFAR-10** Dataset is used for practice for simplicity
*   Part of codes from https://github.com/kuangliu/pytorch-cifar

In [4]:
!nvidia-smi # Make Sure you are using GPU

Mon Jan 25 02:31:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **ResNet Model Definition**
* In this **Tutorial**, **ResNet-18** architecture is utilized since relatively simple dataset is used, **CIFAR-10**.


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.backends.cudnn as cudnn
import torch.optim as optim

import os

# ResNet Architecture consists of small building blocks with skip connection.
# Thus, it is useful to define such block as an independent module.
class BasicBlock(nn.Module):
    """ There are two types of Block in original resnet paper.
    Since we focus on simple CIFAR10 dataset, we are going to utilize basic blocks.

    There's detailed explanation on another type of Block in the paper
    which is for making network deeper with fewer parameters, so
    I highly recommand you to read the original paper :)
    """
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1: # In case of downsizing
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # Skip Connection
        out = F.relu(out)
        return out

# ResNet Implementation
# Especially ResNet18 which is the lightest model in resnet family utilized
# since we are dealing with CIFAR 10 which is relatively small dataset.
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

### **Hyper Parameter Setting**

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu" # whether using gpu or cpu

model = ResNet18() # model assignment
model.to(device) # mapping model weight & bias into gpu memory
model = torch.nn.DataParallel(model) # used for parallel setting

cudnn.benchmark = True # using cudnn which optimizes the algorithm

learning_rate = 0.01
batch_size = 128
max_epoch = 10

model_path = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss() # simple cross-entropy is utilized
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Adam optimizer utilized

* **torchsummary** is package for visualizing pytorch model.
* Layers, Number of parameters can be viewed through this.

In [7]:
import torchsummary

torchsummary.summary(model.cuda(), (3, 32, 32)) # CIFAR-10 Image has 32 x 32 x 3 shape

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

### **Training and Evaluation function definition**

In [8]:
def train(epoch, max_epoch):
    print(f"Train Epoch [{epoch}/{max_epoch}]")
    model.train() # Model to train mode

    train_loss = 0
    correct = 0
    total = 0
    acc = 0

    for idx, (x, y) in enumerate(train_dataloader):
        x = x.to(device) # maps data into GPU memory
        y = y.to(device)

        optimizer.zero_grad() # reset gradients in optimizer before calculating the loss

        y_pred = model(x) # model inference
        loss = criterion(y_pred, y) # calculating the loss

        loss.backward() # back-propagation to get cumulative gradients

        optimizer.step() # update model parameters
        train_loss += loss.item()
        _, inference = y_pred.max(1)

        total += x.size(0)
        correct += inference.eq(y).sum().item()

        if idx % 100 == 0:
            print(f"Epoch [{epoch}/{max_epoch}] Batch [{idx}] Train Loss: {loss.item()}")

    acc = 100*correct/total
    print(f"Epoch [{epoch}/{max_epoch}] Train Loss: {train_loss/total} Train Accuracy: {acc}")

def valid(epoch, max_epoch):
    print(f"Valid Epoch [{epoch}/{max_epoch}]")
    model.eval() # Model to evaluation mode

    valid_loss = 0
    correct = 0
    total = 0

    for idx, (x, y) in enumerate(valid_dataloader):
        x = x.to(device) # maps data into GPU memory
        y = y.to(device)

        with torch.no_grad():
            y_pred = model(x) # model inference
            valid_loss += criterion(y_pred, y).item()

            _, inference = y_pred.max(1)

            total += x.size(0)
            correct += inference.eq(y).sum().item()

    acc = 100*correct/total
    print(f"Epoch [{epoch}/{max_epoch}] Valid Loss: {valid_loss/total} Valid Accuracy: {acc}")

    if not os.path.exists('checkpoint'):
        os.mkdir('checkpoint')

    torch.save(model.state_dict(), f'checkpoint/{model_path}')
    print(f"Epoch [{epoch}/{max_epoch}] Valid Model Saved: checkpoint/{model_path}")

# Custom leraning rate scheduler is use.
# Decay learning rate by 10 at epoch 5.
def lr_schedule(optimizer, epoch):
    if epoch == 5:
        lr = learning_rate / 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

### **Data Preparation**
* In this **Tutorial**, we are going to use **torchvision** which contains famous vision dataset, and we will use **CIFAR-10** especially 
* **CIFAR-10** is dataset for classifying image into **10** different categories.
* The **10** different classes represent **airplanes**, **cars**, **birds**, **cats**, **deer**, **dogs**, **frogs**, **horses**, **ships**, and **trucks**. 
* There are **6,000** images of each class.


In [9]:
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
valid_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### **Training**

In [10]:
for epoch in range(0, max_epoch):
    lr_schedule(optimizer, epoch)
    train(epoch, max_epoch)
    valid(epoch, max_epoch)

Train Epoch [0/10]
Epoch [0/10] Batch [0] Train Loss: 2.37066650390625
Epoch [0/10] Batch [100] Train Loss: 1.6077157258987427
Epoch [0/10] Batch [200] Train Loss: 1.5893092155456543
Epoch [0/10] Batch [300] Train Loss: 1.6005460023880005
Epoch [0/10] Train Loss: 0.013065234131813049 Train Accuracy: 38.574
Valid Epoch [0/10]
Epoch [0/10] Valid Loss: 0.011038797342777252 Valid Accuracy: 48.94
Epoch [0/10] Valid Model Saved: checkpoint/resnet18_cifar10.pt
Train Epoch [1/10]
Epoch [1/10] Batch [0] Train Loss: 1.2568929195404053
Epoch [1/10] Batch [100] Train Loss: 1.165838360786438
Epoch [1/10] Batch [200] Train Loss: 1.0081522464752197
Epoch [1/10] Batch [300] Train Loss: 1.0745385885238647
Epoch [1/10] Train Loss: 0.008944134958982468 Train Accuracy: 58.904
Valid Epoch [1/10]
Epoch [1/10] Valid Loss: 0.008684489226341248 Valid Accuracy: 60.83
Epoch [1/10] Valid Model Saved: checkpoint/resnet18_cifar10.pt
Train Epoch [2/10]
Epoch [2/10] Batch [0] Train Loss: 0.9922165274620056
Epoch [2/1